In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)


engine.dispose()

In [2]:
weather_df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [3]:
#Setting Variables
Y = weather_df.apparenttemperature - weather_df.temperature
X = weather_df[['humidity', 'windspeed']]

In [4]:
#Running regression model

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Sat, 17 Aug 2019   Prob (F-statistic):               0.00
Time:                        22:32:34   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Are the estimated coefficients statistically significant? 
Both humidity and windspeed have p-values of zero, so they are statistically significant.  

#### Are the signs of the estimated coefficients in line with your previous expectations? 
No.  My first impression was that if there were an increase in humidity, there would be a corresponding increase in the target.  The sign of windspeed makes sense - if there is more wind, the target (apparent temp - temp) will decrease.    

#### Interpret the estimated coefficients. What are the relations between the target and the explanatory variables? 
The constant/bias term is 2.4381.  As the humidity increases by one point, there is a 3.0292 decrease in the target.  And as windspeed increases by one point, there is a 0.1193 decrease in the target. 

In [5]:
#Setting Variables
weather_df['humidity_windspeed'] = weather_df.humidity * weather_df.windspeed
Y = weather_df.apparenttemperature - weather_df.temperature
X = weather_df[['humidity', 'windspeed', 'humidity_windspeed']]

In [6]:
#Running regression model

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Sat, 17 Aug 2019   Prob (F-statistic):               0.00
Time:                        22:32:34   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0839      0.033      2.511      0.012       0.018       0.149
humidity               0.1775      0.043      4.133      0.000       0.093       0.262
windspeed              0.0905      0.002     36.797      0.000       0.086       0.095
humidity_windspeed    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Are the coefficients statistically significant? 
Yes, all of the coefficients have a p-value of zero and are statistically significant. 

#### Did the signs of the estimated coefficients for humidity and windspeed change? 
Yes, both signs changed from negative to positive. 

#### Interpret the estimated coefficients.
Temperature = B0 + B1*Humidity + B2*Windspeed + B3*Humidity*Windspeed<br><br>
Temperature = 0.0839 + 0.1775*Humidity + 0.0905*Windspeed -.2971*Humidity*Windspeed<br><br>

The constant/bias term is 0.0839.  As humidity increases one point, the target increases by 0.1775 (when windspeed is 0), and as the windspeed increases by one point, the target increases by 0.0905 (when humidity is 0). For the humidity/windspeed interaction:

A one point increase in windspeed results in a 0.1775 - .2971* humidity point change in temperature.  <br>
A one point increase in humidity results in a 0.0905 - .2971x* windspeed point change in temperature.